# Project - Airline AI Assistant

We'll now bring together what we've learned to make an AI Customer Support assistant for an Airline

In [63]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr
from anthropic import Anthropic


In [75]:
# Initialization

load_dotenv(override=True)

openai_api_key = os.getenv('OPENAI_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
    

#MODEL = "gpt-4o-mini"
#openai = OpenAI()

# As an alternative, if you'd like to use Ollama instead of OpenAI
# Check that Ollama is running for you locally (see week1/day2 exercise) then uncomment these next 2 lines
#MODEL = "deepseek-r1"
openai = OpenAI(base_url='http://localhost:11434/v1', api_key='ollama')

MODEL = "claude-3-haiku-20240307"
claude_key = os.getenv('CLAUDE_API_KEY')
anthropic_client = Anthropic(api_key=claude_key)


OpenAI API Key exists and begins sk-proj-


**Anthropic Test**

In [76]:
our_first_message = anthropic_client.messages.create(
    model=MODEL,
    max_tokens=1000,
    messages=[
        {"role": "user", "content": "Hi there! Please write me a haiku about a pet chicken"}
    ]
)

print(our_first_message.content[0].text)

Fluffy feathers gleam,
Clucking softly in the yard,
Loyal pet chicken.


In [58]:
system_message = "You are a concise, helpful assistant for an Airline called FlightAI. "
system_message += "Give short, courteous answers, no more than 1 sentence. Respond only with the final answer, no explanations, and do not show your reasoning steps or thinking. "
system_message += "Always be accurate. If you don't know the answer, say so."

In [ ]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages)
    return response.choices[0].message.content
response = chat("What is the price of a ticket from London to Berlin? Do not show me your reasoning", [])
print(response)
gr.ChatInterface(fn=chat, type="messages").launch()

<think>
Hmm, the user is asking about the ticket price from London to Berlin without wanting any explanation or reasoning shown in my response (according to system instructions). 

I only know information about flights between major airlines' home cities according to my knowledge cutoff. My response should stay concise and accurate by using brackets for standard pricing ranges.

London to Berlin is a common flight route that might interest travelers planning trips across Europe through Germany, perhaps business people flying from UK base or tourists visiting historic sites in Berlin.
</think>
Approximately £100-£300 [depending heavily on class, time of booking, season and airline]
* Running on local URL:  http://127.0.0.1:7887
* To create a public link, set `share=True` in `launch()`.


Anthropic Version of simple chat

In [ ]:
# This function looks rather simpler than the one from my video, because we're taking advantage of the latest Gradio updates

def chat(message, history):
    history = [{k: v for k, v in d.items() if k not in ("metadata", "options")} for d in history]
    messages = history + [{"role": "user", "content": message}]
    response = anthropic_client.messages.create(model=MODEL, max_tokens = 1000,system=system_message, messages=messages)
    return response.content[0].text


gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7900
* To create a public link, set `share=True` in `launch()`.


History: []
Messages: [{'role': 'user', 'content': 'hello'}]
History: [{'role': 'user', 'metadata': None, 'content': 'hello', 'options': None}, {'role': 'assistant', 'metadata': None, 'content': 'Hello.', 'options': None}]
Messages: [{'role': 'user', 'content': 'hello'}, {'role': 'assistant', 'content': 'Hello.'}, {'role': 'user', 'content': 'Help me find flights to Tokyo'}]


## Tools

Tools are an incredibly powerful feature provided by the frontier LLMs.

With tools, you can write a function, and have the LLM call that function as part of its response.

Sounds almost spooky.. we're giving it the power to run code on our machine?

Well, kinda.

In [85]:
# Let's start by making a useful function

ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(destination_city):
    print(f"Tool get_ticket_price called for {destination_city}")
    city = destination_city.lower()
    return ticket_prices.get(city, "Unknown")

In [86]:
get_ticket_price("London")  # Example call to test the function

Tool get_ticket_price called for London


'$799'

In [ ]:
# There's a particular dictionary structure that's required to describe our function:
padfadfdsdsfadfasfasdfsadfsadfasdfasdfadsfasdfasdfasdfsdghhbcvnxcvbzfserice_function = {
    "name": "get_ticket_price",
    "description": "Get the price of a return ticket to the destination city. Call this whenever you need to know the ticket price, for example when a customer asks 'How much is a ticket to this city'",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {
                "type": "string",
                "description": "The city that the customer wants to travel to",
            },
        },
        "required": ["destination_city"],
        "additionalProperties": False
    }
}

In [88]:
# And this is included in a list of tools:

tools = [{"type": "function", "function": price_function}]

## Getting OpenAI to use our Tool

There's some fiddly stuff to allow OpenAI "to call our tool"

What we actually do is give the LLM the opportunity to inform us that it wants us to run the tool.

Here's how the new chat function looks:

In [89]:
def chat(message, history):
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)

    if response.choices[0].finish_reason=="tool_calls":
        message = response.choices[0].message
        response, city = handle_tool_call(message)
        print(f"Handle_tool_call: {response}") 
       # result = handle_tool_call(message)
        messages.append(message)
        messages.append(response)
        print(messages)
        response = openai.chat.completions.create(model=MODEL, messages=messages)
    
    return response.choices[0].message.content

In [90]:
# We have to write that function handle_tool_call:

def handle_tool_call(message):
    tool_call = message.tool_calls[0]
    arguments = json.loads(tool_call.function.arguments)
    city = arguments.get('destination_city')
    price = get_ticket_price(city)
    response = {
        "role": "tool",
        "content": json.dumps({"destination_city": city,"price": price}),
        "tool_call_id": tool_call.id
    }
    print(f"Tool call response: {response}")
    return response, city

    #return response

In [91]:
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7887
* To create a public link, set `share=True` in `launch()`.


Traceback (most recent call last):
  File "/Users/jesusgarcia/.pyenv/versions/3.11.12/envs/llms/lib/python3.11/site-packages/gradio/queueing.py", line 625, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jesusgarcia/.pyenv/versions/3.11.12/envs/llms/lib/python3.11/site-packages/gradio/route_utils.py", line 322, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jesusgarcia/.pyenv/versions/3.11.12/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 2147, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jesusgarcia/.pyenv/versions/3.11.12/envs/llms/lib/python3.11/site-packages/gradio/blocks.py", line 1663, in call_function
    prediction = await fn(*processed_input)
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jesusgarcia/.pyenv/versions/3